In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from cl.eval import EvalReader as EPPS
import seaborn as sns
import numpy as np
import os


In [ ]:
printn = print
def print1(*arg, **karg):
    pass

def common_parse(name, add):
    if name.find("pertask")>=0:
        _, tid, fold = name.split("_")
        name = "{} on fold {}".format(tid, fold)
    return "Result of Task {} {}:".format(name, add)
def listdct_to_dct(lst, kname, vname):
    ret = {}
    for d in lst:
        ret[tuple(d[kname])] = d[vname]
    return ret
def parse_pairwise_ic(name, measure):
    tname = common_parse(name, "Pairwise Inconsistency")
    print(tname)
    pairwise = listdct_to_dct(measure["inconsist_pairwise"], kname="compare", vname="consistency")
    idx = measure["index"]
    l = len(idx)
    _idx_name = ["Model {}".format(i) for i in idx]
    _df = pd.DataFrame(index = _idx_name[:-1], columns = _idx_name[1:]).fillna("")
    for i in range(l):
        for j in range(i+1, l):
            _df.loc[_idx_name[i], _idx_name[j]] = pairwise[(idx[i],idx[j])]
    #_df.style.set_caption(tname)
    print(_df)
    return _df

def parse_acc(name, measure):
    print(common_parse(name, "Accuracy Per Step"))
    for i in range(len(measure["index"])):
        print(measure["acc_list"][i], end="\t")
    print("avg acc", np.mean(measure["acc_list"]))
    return np.mean(measure["acc_list"])#measure["acc_list"][0]#np.mean(measure["acc_list"])#measure["acc_list"][-1]#

def parse_total(name, measure):
    print(common_parse(name, "Inconsistency Together"))
    print(measure["inconsist_tot"])
    return measure["inconsist_tot"]

def filter_lst(lst, s):
    rst = []
    for _s in lst:
        if s[0] == "-":
            if _s.find(s)==-1:
                rst.append(_s)
        else:
            if _s.find(s)>=0:
                rst.append(_s)
    return rst

def sname(lst):
    global common_filter
    lst += common_filter
    dn = os.listdir("results")
    for s in lst:
        dn = filter_lst(dn, s)
    if len(dn) > 1:
        print("multiple result" + str(dn))
    if len(dn) == 0:
        assert False
    mx_sz = 1000
    mx_id = 0
    for idx, _d in enumerate(dn):
        if len(_d) < mx_sz:
            mx_sz = len(_d)
            mx_id = idx
    printn(dn[mx_id])
    return os.path.join("results", dn[mx_id], "Seed0")

In [ ]:
def read_result(evaluator, bname):
    global rec
    e = evaluator
    print("="*50)
    rec[bname] = []
    print(bname)
    for name in e.names:
        parse_pairwise_ic(name, e._measure_pairwise(name))
        print()
        acc = parse_acc(name,e._measure_acc(name))
        print()
        tot = parse_total(name, e._measure([name]))
        true_name = "_".join(name.split("_")[-2:])
        rec[bname].append((true_name,tot,acc))
        print(tot)
def draw_pic(picname, names):
    
    #names = [ ]#, "lwf_scratch", "ewc_scratch", "vanilla_scratch",]
    tasks = []
    vals = []
    comps = []
    accs = []
    print(rec)
    for keyword in ["train", "test", "val"]:
        for name in names:
            for v in rec[name]:
                task, val, acc = v
                if task.find(keyword)<0:
                    continue
                comps.append(name)
                tasks.append(task)
                accs.append(acc)
                vals.append(val)
    plt.figure(figsize=(25,10))
    plt.subplot(1,2,1)
    df = pd.DataFrame({"Task":tasks, "Method":comps, "Inconsistency":vals, "Acc":accs})
    pd.melt(df, id_vars=['Task'], value_vars=['Inconsistency'])
    sns.barplot(x = 'Task', y = 'Inconsistency', data = df, hue="Method")
    plt.ylabel('Inconsistency')
    plt.title('Inconsistency over different baselins')
    #plt.savefig("Inconsistency_{}.png".format(picname))
    #plt.show()
    #plt.figure(figsize=(10,10))
    plt.subplot(1,2,2)
    df = pd.DataFrame({"Task":tasks, "Method":comps, "Inconsistency":vals, "Acc":accs})
    pd.melt(df, id_vars=['Task'], value_vars=['Acc'])
    sns.barplot(x = 'Task', y = 'Acc', data = df, hue="Method")
    plt.ylabel('Acc')
    plt.title('Acc over different baselins')
    plt.savefig("Acc_{}.png".format(picname))
    plt.show()
def draw_pic_interpolate(picname, names):
    
    #names = [ ]#, "lwf_scratch", "ewc_scratch", "vanilla_scratch",]
    tasks = []
    vals = []
    comps = []
    accs = []
    print(rec)
    for keyword in ["test"]:
        for name in names:
            for v in rec[name]:
                task, val, acc = v
                #print(task)
                if task.find(keyword)<0 or task.find("0_")<0:
                    continue
                comps.append(name)
                tasks.append(task)
                accs.append(acc)
                vals.append(val)
    plt.figure(figsize=(10,10))
    #plt.subplot(1,2,1)
    #df = pd.DataFrame({"Task":tasks, "Method":comps, "Inconsistency":vals, "Acc":accs})
    plt.plot(mid_lst, vals, label="inconsistency")
    for idx, (a,b) in enumerate(zip(mid_lst, vals)): 
        if idx % 2==0:
            plt.text(a, b, "{:.2f}".format(b))
    plt.plot(mid_lst, accs, label="accuracy")
    for idx, (a,b) in enumerate(zip(mid_lst, accs)): 
        if idx % 2==0:
            plt.text(a, b, "{:.2f}".format(b))    
    #sns.barplot(x = 'Task', y = 'Inconsistency', data = df, hue="Method")
    #plt.ylabel('Inconsistency')
    plt.title('Inconsistency and accuracy with interpolation')
    plt.legend()
    plt.savefig("Interpolation_{}.png".format(picname))
    plt.show()

In [ ]:
epps = EPPS()
case = 5
if case  == 1:
    common_filter = ["cifar10", "small", "ResNet18_1.0e-03","CTdata", "FixInit"]#"CvgS200"]"small",
    for i in range(1):
        rec = {}
        epps.load(sname(["scratch"]))#, "small_"]
        epps.set_order("sequential")
        read_result(epps, "vanilla_scratch")

        epps.load(sname(["#hist_avg"]))#, "small_"]
        epps.set_order("sequential")
        read_result(epps, "hist_ensemble")
        
        epps.load(sname(["vanilla"]))#, "small_"]
        epps.set_order("sequential")
        read_result(epps, "vanilla_finetune")

        epps.load(sname(["ensemble_5_", "snapshot"]))#, "small_"]
        epps.set_order("sequential")
        read_result(epps, "Ensemble_Snapshot")
        
        epps.load(sname(["#Aug_CF_R"]))
        epps.set_order("sequential")
        read_result(epps, "vanilla_Aug")

        epps.load(sname(["#Aug_CF_ADV_R"]))
        epps.set_order("sequential")
        read_result(epps, "vanilla_Aug_ADV")

        epps.load(sname(["#Aug_ADV"]))
        epps.set_order("sequential")
        read_result(epps, "vanilla_Adv")
        
        #epps.load(sname(["scratch"]))
        #epps.set_order("sequential")
        #read_result(epps, "vanilla_scratch")
        
        epps.load(sname(["#lwf1.00e+00_DS"]))
        epps.set_order("sequential")
        read_result(epps, "lwf")

        epps.load(sname(["#lwf1.00e+00_unsup"]))
        epps.set_order("sequential")
        read_result(epps, "lwf_Unsupervised_SameDist")

        epps.load(sname(["#lwf1.00e+00#corrset"]))
        epps.set_order("sequential")
        read_result(epps, "lwf_Correct_Subset")  

        epps.load(sname(["#ewc"]))
        epps.set_order("sequential")
        read_result(epps,"ewc")
        draw_pic("Data_Inc_seed_{}".format(i),\
                 names=[ "vanilla_scratch", "vanilla_finetune","hist_ensemble", "Ensemble_Snapshot", "vanilla_Aug", "vanilla_Adv", "vanilla_Aug_ADV", "ewc", "lwf", "lwf_Unsupervised_SameDist", "lwf_Correct_Subset"])   
elif case == 2:
    common_filter = ["cifar10", "ResNet18_1.0e-03","CTdata", "FixInit", "-small"]#"CvgS200"]"small",
    for i in range(1):
        rec = {}
        epps.load(sname(["scratch"]))#, "small_"]
        epps.set_order("sequential")
        read_result(epps, "vanilla_scratch")
        
        epps.load(sname(["vanilla"]))#, "small_"]
        epps.set_order("sequential")
        read_result(epps, "vanilla_finetune")
        
        epps.load(sname(["#Aug_CF_R"]))
        epps.set_order("sequential")
        read_result(epps, "vanilla_Aug")

        epps.load(sname(["#Aug_CF_ADV_R"]))
        epps.set_order("sequential")
        read_result(epps, "vanilla_Aug_ADV")

        epps.load(sname(["#Aug_ADV"]))
        epps.set_order("sequential")
        read_result(epps, "vanilla_Adv")
        
        #epps.load(sname(["scratch"]))
        #epps.set_order("sequential")
        #read_result(epps, "vanilla_scratch")
        
        epps.load(sname(["#lwf1.00e+00_DS"]))
        epps.set_order("sequential")
        read_result(epps, "lwf")

        #epps.load(sname(["#lwf1.00e+00_unsup"]))
        #epps.set_order("sequential")
        #read_result(epps, "lwf_Unsupervised_SameDist")

        epps.load(sname(["#lwf1.00e+00#corrset"]))
        epps.set_order("sequential")
        read_result(epps, "lwf_Correct_Subset")  

        epps.load(sname(["#ewc"]))
        epps.set_order("sequential")
        read_result(epps,"ewc")
        draw_pic("Data_Inc_seed_{}".format(i),\
                 names=[ "vanilla_scratch", "vanilla_finetune", "vanilla_Aug", "vanilla_Adv", "vanilla_Aug_ADV", "ewc","lwf_Correct_Subset", "lwf",])# "lwf_Unsupervised_SameDist", "lwf_Correct_Subset"])      
elif case ==3:
    for i in range(1):
        rec = {}
        #epps.load("results/cifar10_5_domain_inc")
        epps.load("results/cifar10_ResNet18_1.0e-02_#vanilla_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        read_result(epps, "vanilla_finetune")
        epps.load("results/cifar10_ResNet18_1.0e-02_#lwf1.00e+00_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        read_result(epps, "lwf")
        epps.load("results/cifar10_ResNet18_1.0e-02_#ewc5.00e+03_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        read_result(epps,"ewc")
        epps.load("results/cifar10_ResNet18_1.0e-02_#scratch_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        read_result(epps, "vanilla_scratch")
        """epps.load("results/cifar10_ResNet18_1.0e-02_#lwf1.00e+00#scratch_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        read_result(epps, "lwf_scratch")
        epps.load("results/cifar10_ResNet18_1.0e-02_#ewc5.00e+03#scratch_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        read_result(epps,"ewc_scratch")"""
        draw_pic("Domain_Inc_seed_{}".format(i),\
                 names=["vanilla_scratch", "vanilla_finetune", "ewc", "lwf"]) 
elif case == 4:
    for i in range(1):
        rec = {}
        #epps.load("results/cifar10_5_domain_inc")
        epps.load("results/cifar10_ResNet18_1.0e-02_#vanilla_DScifar10_CIMmask_CTdata_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        epps.set_order("sequential")
        read_result(epps, "vanilla_finetune")
        epps.load("results/cifar10_ResNet18_1.0e-02_#lwf1.00e+00_DScifar10_CIMmask_CTdata_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        epps.set_order("sequential")
        read_result(epps, "lwf")
        epps.load("results/cifar10_ResNet18_1.0e-02_#scratch_DScifar10_CIMmask_CTdata_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        epps.set_order("sequential")
        read_result(epps, "vanilla_scratch")
        epps.load("results/cifar10_ResNet18_1.0e-02_#ewc5.00e+03_DScifar10_CIMmask_CTdata_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        epps.set_order("sequential")
        read_result(epps,"ewc")
        """epps.load("results/cifar10_ResNet18_1.0e-02_#lwf1.00e+00#scratch_DScifar10_CIMmask_CTdata_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        read_result(epps, "lwf_scratch")
        epps.load("results/cifar10_ResNet18_1.0e-02_#ewc5.00e+03#scratch_DScifar10_CIMmask_CTdata_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed{}".format(i))
        read_result(epps,"ewc_scratch")"""
        draw_pic("Data_Inc_seed_{}".format(i),\
                 names=["vanilla_scratch", "vanilla_finetune", "ewc", "lwf"])
elif case == 5:
        
    common_filter = ["cifar10", "ResNet18_1.0e-03","CTdata", "FixInit", "small"]#"CvgS200"]"small",
    rec = {}
    mid_lst = range(1,20)
    epps.load(sname(["interpolate"]))#, "small_"]
    for i in range(1,20):
        epps.set_order("set", val=[0,i,20])
        read_result(epps, str(i))
    draw_pic_interpolate("Interpolate_Data_Inc_seed_0", [str(i) for i in mid_lst])

In [ ]:
from cl.utils import get_fixed_random_generator
import itertools
print(list(itertools.islice(get_fixed_random_generator(10,1), 5)))

In [ ]:
print(epps.names)
print(epps.hist_version["pertask_0_test"][:10,:10])